In [1]:
import os
import numpy as np
import pandas as pd
import folium
from folium import plugins
import geopandas
import argparse
import glob
import gpxpy
import geojson
import webbrowser
import numpy as np
import matplotlib.cm as cm
from scipy.signal import medfilt


In [2]:
# functions
def rgb2hex(c):
    hexc = '#%02x%02x%02x'%(int(c[0]*255), int(c[1]*255), int(c[2]*255))
    return(hexc)

def calc_dist_from_coords(p1, p2): # distance between p1 and p2 [lat,lon] (in deg)
    lat1 = np.radians(p1[0])
    lat2 = np.radians(p2[0])
    lon1 = np.radians(p1[1])
    lon2 = np.radians(p2[1])

    delta_lat = lat2-lat1
    delta_lon = lon2-lon1

    # Haversine formula
    a = np.power(np.sin(delta_lat/2.0), 2)+np.cos(lat1)*np.cos(lat2)*np.power(np.sin(delta_lon/2.0), 2)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    dist = 6371e3*c

    return(dist)

def calc_dist_from_coordsPoint2Line(p0, p1, p2): # distance from p0 to line defined by p1 and p2 [lat,lon] (in deg)
    # Mercator projection
    P0 = np.array([np.radians(p0[1]), np.arcsinh(np.tan(np.radians(p0[0])))])*6371e3
    P1 = np.array([np.radians(p1[1]), np.arcsinh(np.tan(np.radians(p1[0])))])*6371e3
    P2 = np.array([np.radians(p2[1]), np.arcsinh(np.tan(np.radians(p2[0])))])*6371e3

    # distance from point to line
    dist = abs((P2[1]-P1[1])*P0[0]-(P2[0]-P1[0])*P0[1]+P2[0]*P1[1]-P2[1]*P1[0])/np.sqrt(np.power(P2[1]-P1[1], 2)+np.power(P2[0]-P1[0], 2)) # (from https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line#Line_defined_by_two_points)

    return(dist)

def RDP(data, epsilon): # Ramer–Douglas–Peucker algorithm
    if epsilon <= 0:
        return(data)

    dist_max = 0
    index = 0

    for i in np.arange(1, data.shape[0]):
        dist = calc_dist_from_coordsPoint2Line(data[i, :2], data[0, :2], data[-1, :2]) # needs a 2D projection, does not work with cross-track distance

        if dist > dist_max:
            index = i
            dist_max = dist

    if dist_max > epsilon:
        tmp1 = RDP(data[:index+1, :], epsilon)
        tmp2 = RDP(data[index:, :], epsilon)

        data_new = np.vstack((tmp1[:-1], tmp2))
    else:
        data_new = np.vstack((data[0, :], data[-1, :]))

    return(data_new)



In [11]:
data_input_raw = 'data_input_raw' 
data_processed_gpx = 'data_processed_gpx' 
data_geojson = 'data_geojson'
visualize = False
#SI_units = True

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#gpx_file_temp = os.path.join(dir_work, 'data_input_gpx/Afternoon_Run55.gpx')
#print(os.path.isfile(gpx_file_temp))

ingest_file_list = glob.glob(os.path.join(data_input_raw, '*.gpx'))
n_files = len(ingest_file_list)
print('found %s files to process ' %(n_files))                              
#print('found %s files' %(n_files))                              
#geojson_file = os.path.join(data_geojson, '2020-03-22_15-06.geojson')
#os.path.isfile(geojson_file)



found 33 files to process 


In [4]:
f = 0
#for f in range(0, n_files, 1):
for f in range(0, 4, 1):
    #print('  processing %s of % n_files ' %(f, n_files))                              
    gpx_file_temp = ingest_file_list[f]

    lat_lon_data = []
    elevation_data = []
    dt_data = []
    timestamp_data = []

    # read GPX file
    with open(gpx_file_temp, 'r') as file:
        gpx = gpxpy.parse(file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    lat_lon_data.append([point.latitude, point.longitude])
                    elevation_data.append(point.elevation)
                    timestamp_data.append(point.time.timestamp()) # convert time to timestamps (s)
                    dt_data.append(point.time) # convert time to timestamps (s)

    lat_lon_data   = np.array(lat_lon_data)  # [deg, deg]
    elevation_data = np.array(elevation_data) # [m]
    timestamp_data = np.array(timestamp_data) # [s]

    n_points = len(timestamp_data)
    print('  read %s points ' %(n_points))    
    # calculate trackpoints distance, slope, speed and power
    distance_data = np.zeros([n_points]) # [m]
    slope_data    = np.zeros([n_points]) # [%]
    speed_data    = np.zeros([n_points]) # [m/s]

    for i in np.arange(1, n_points):
        distance_data[i] = calc_dist_from_coords(lat_lon_data[i-1, :], lat_lon_data[i, :])
        delta_elevation = elevation_data[i]-elevation_data[i-1]
        if (distance_data[i] > 0):
            slope_data[i] = delta_elevation/distance_data[i]
        distance_data[i] = np.sqrt(np.power(distance_data[i], 2.0)+np.power(delta_elevation, 2.0)) # recalculate distance to take slope into account
    for i in np.arange(1, timestamp_data.shape[0]):
        if (timestamp_data[i] != timestamp_data[i-1]):
            speed_data[i] = distance_data[i]/(timestamp_data[i]-timestamp_data[i-1])

    # filter speed and slope data (default Strava filters)
    slope_data = medfilt(slope_data, 5)
    speed_data = medfilt(speed_data, 5)

    n_points = len(slope_data)
    print('  read %s points ' %(n_points))    
    
    use_RDP = True
    # use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
    if use_RDP:
        epsilon = 1 # [m]
        tmp = np.hstack((lat_lon_data, np.arange(0, lat_lon_data.shape[0]).reshape((-1, 1)))) # hack
        tmp_new = RDP(tmp, epsilon) # remove trackpoints less than epsilon meters away from the new track
        index = tmp_new[:, 2].astype(int) # hack
        lat_lon_data = lat_lon_data[index, :]
        elevation_data = elevation_data[index]
        timestamp_data = timestamp_data[index]
        distance_data = distance_data[index]
        slope_data = slope_data[index]
        speed_data = speed_data[index]
    slope_data = abs(slope_data*100) # decimal to %

    n_points = len(slope_data)
    print('  read %s points ' %(n_points))    
    
    # create GeoJSON feature collection
    features = []
    for i in np.arange(1, n_points):
        line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
        feature = geojson.Feature(geometry=line, properties={'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
        features.append(feature)

    feature_collection = geojson.FeatureCollection(features)

    file_name = os.path.basename(gpx_file_temp.strip('.gpx'))
    # write geojson file
    geojson_write_file = gpx_file_temp.replace(file_name,dt_data[0].strftime('%Y-%m-%d_%H-%M')).replace(data_input_raw,data_geojson).replace('.gpx','.geojson')        
    print('  geojson_write_file is %s ' %(geojson_write_file))
    with open(geojson_write_file, 'w') as file:
        geojson.dump(feature_collection, file)

    # rename and archive gpx file 
    gpx_file_name_archive = gpx_file_temp.replace(file_name,dt_data[0].strftime('%Y-%m-%d_%H-%M')).replace(data_input_raw,data_processed_gpx)        
    print('  gpx_file_name_archive is %s ' %(gpx_file_name_archive))
    if (' ' in gpx_file_temp):
        temp_command = 'mv -f "'+gpx_file_temp+'" '+gpx_file_name_archive
    else:
        temp_command = 'mv -f '+gpx_file_temp+' '+gpx_file_name_archive

    print('  temp_command is %s ' %(temp_command))    
    os.system(temp_command)

    
    

  read 0 points 
  read 0 points 


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

use_RDP = True
# use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
if (use_RDP):
    epsilon = 1 # [m]
    tmp = np.hstack((lat_lon_data, np.arange(0, n_points).reshape((-1, 1)))) # hack
    tmp_new = RDP(tmp, epsilon) # remove trackpoints less than epsilon meters away from the new track
    index = tmp_new[:, 2].astype(int) # hack
    lat_lon_data   = lat_lon_data  [index,:]
    elevation_data = elevation_data[index]
    timestamp_data = timestamp_data[index]
    distance_data  = distance_data [index]
    slope_data     = slope_data    [index]
    speed_data     = speed_data    [index]

n_points = len(slope_data)
print('  read %s points ' %(n_points))    
    
# convert units
if use_SI:
    speed_data = speed_data*3.6 # m/s to km/h
else:
    speed_data = speed_data*2.236936 # m/s to mph

slope_data = abs(slope_data*100) # decimal to %

# create GeoJSON feature collection
features = []
for i in np.arange(1, n_points):
    #print('    processing %s of %s points ' %(i, n_points))    
    #print('    %s, %s, %s, %s ' %(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0], lat_lon_data[i, 1], lat_lon_data[i, 0]))    
    try:
        line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
        feature = geojson.Feature(geometry=line, properties = {'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
        features.append(feature)
    except:
        print('    ERROR %s of %s points ' %(i, n_points))    

feature_collection = geojson.FeatureCollection(features)


In [12]:
f = 0

gpx_file_temp = ingest_file_list[f]
print(gpx_file_temp)
lat_lon_data = []
elevation_data = []
dt_data = []
timestamp_data = []

# read GPX file
with open(gpx_file_temp, 'r') as file:
    gpx = gpxpy.parse(file)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lat_lon_data.append([point.latitude, point.longitude])
                elevation_data.append(point.elevation)
                timestamp_data.append(point.time.timestamp()) # convert time to timestamps (s)
                dt_data.append(point.time) # convert time to timestamps (s)

lat_lon_data   = np.array(lat_lon_data)  # [deg, deg]
elevation_data = np.array(elevation_data) # [m]
timestamp_data = np.array(timestamp_data) # [s]

    
    

data_input_raw/Afternoon_Run (1).gpx


In [9]:
print(lat_lon_data)

[]


In [6]:
n_points = len(timestamp_data)
print('  read %s points ' %(n_points))    
# calculate trackpoints distance, slope, speed and power
distance_data = np.zeros([n_points]) # [m]
slope_data    = np.zeros([n_points]) # [%]
speed_data    = np.zeros([n_points]) # [m/s]

for i in np.arange(1, n_points):
    distance_data[i] = calc_dist_from_coords(lat_lon_data[i-1, :], lat_lon_data[i, :])
    delta_elevation = elevation_data[i]-elevation_data[i-1]
    if (distance_data[i] > 0):
        slope_data[i] = delta_elevation/distance_data[i]
    distance_data[i] = np.sqrt(np.power(distance_data[i], 2.0)+np.power(delta_elevation, 2.0)) # recalculate distance to take slope into account
for i in np.arange(1, timestamp_data.shape[0]):
    if (timestamp_data[i] != timestamp_data[i-1]):
        speed_data[i] = distance_data[i]/(timestamp_data[i]-timestamp_data[i-1])

# filter speed and slope data (default Strava filters)
slope_data = medfilt(slope_data, 5)
speed_data = medfilt(speed_data, 5)

n_points = len(slope_data)
print('  read %s points ' %(n_points))    



  read 0 points 
  read 0 points 


In [ ]:
use_RDP = True
# use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
if use_RDP:
    epsilon = 1 # [m]
    tmp = np.hstack((lat_lon_data, np.arange(0, lat_lon_data.shape[0]).reshape((-1, 1)))) # hack
    tmp_new = RDP(tmp, epsilon) # remove trackpoints less than epsilon meters away from the new track
    index = tmp_new[:, 2].astype(int) # hack
    lat_lon_data = lat_lon_data[index, :]
    elevation_data = elevation_data[index]
    timestamp_data = timestamp_data[index]
    distance_data = distance_data[index]
    slope_data = slope_data[index]
    speed_data = speed_data[index]
slope_data = abs(slope_data*100) # decimal to %

n_points = len(slope_data)
print('  read %s points ' %(n_points))    

# create GeoJSON feature collection
features = []
for i in np.arange(1, n_points):
    line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
    feature = geojson.Feature(geometry=line, properties={'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
    features.append(feature)

feature_collection = geojson.FeatureCollection(features)



In [13]:
file_name = os.path.basename(gpx_file_temp.strip('.gpx'))
print(file_name)



Afternoon_Run (1)


In [14]:
# write geojson file
geojson_write_file = gpx_file_temp.replace(file_name,dt_data[0].strftime('%Y-%m-%d_%H-%M')).replace(data_input_raw,data_geojson).replace('.gpx','.geojson')        
print('  geojson_write_file is %s ' %(geojson_write_file))


  geojson_write_file is data_geojson/2020-02-02_00-37.geojson 


In [ ]:
with open(geojson_write_file, 'w') as file:
    geojson.dump(feature_collection, file)



In [15]:
# rename and archive gpx file 
gpx_file_name_archive = gpx_file_temp.replace(file_name,dt_data[0].strftime('%Y-%m-%d_%H-%M')).replace(data_input_raw,data_processed_gpx)        
print('  gpx_file_name_archive is %s ' %(gpx_file_name_archive))


  gpx_file_name_archive is data_processed_gpx/2020-02-02_00-37.gpx 


In [17]:
' ' in gpx_file_temp




True

In [19]:
temp_command = 'mv -f "'+gpx_file_temp+'" '+gpx_file_name_archive
print('  temp_command is %s ' %(temp_command))    


  temp_command is mv -f "data_input_raw/Afternoon_Run (1).gpx" data_processed_gpx/2020-02-02_00-37.gpx 


In [ ]:
os.system(temp_command)
